In [1]:
import numpy as np
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ddf = dd.read_parquet("D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-01.parquet", npartitions=1)

In [3]:
ddf.head().T

,0,1,2,3,4
VendorID,1,1,2,2,2
tpep_pickup_datetime,2022-01-01 00:35:40,2022-01-01 00:33:43,2022-01-01 00:53:21,2022-01-01 00:25:21,2022-01-01 00:36:48
tpep_dropoff_datetime,2022-01-01 00:53:29,2022-01-01 00:42:07,2022-01-01 01:02:19,2022-01-01 00:35:23,2022-01-01 01:14:20
passenger_count,2.0,1.0,1.0,1.0,1.0
trip_distance,3.8,2.1,0.97,1.09,4.3
RatecodeID,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N
PULocationID,142,236,166,114,68
DOLocationID,236,42,166,68,163
payment_type,1,1,1,2,1


In [4]:
ddf = ddf.set_index(ddf.tpep_pickup_datetime, sorted=True)

In [5]:
ddf.head().T

tpep_pickup_datetime,2022-01-01 00:35:40,2022-01-01 00:33:43,2022-01-01 00:53:21,2022-01-01 00:25:21,2022-01-01 00:36:48
VendorID,1,1,2,2,2
tpep_pickup_datetime,2022-01-01 00:35:40,2022-01-01 00:33:43,2022-01-01 00:53:21,2022-01-01 00:25:21,2022-01-01 00:36:48
tpep_dropoff_datetime,2022-01-01 00:53:29,2022-01-01 00:42:07,2022-01-01 01:02:19,2022-01-01 00:35:23,2022-01-01 01:14:20
passenger_count,2.0,1.0,1.0,1.0,1.0
trip_distance,3.8,2.1,0.97,1.09,4.3
RatecodeID,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N
PULocationID,142,236,166,114,68
DOLocationID,236,42,166,68,163
payment_type,1,1,1,2,1


In [6]:
ddf.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,
2022-01-31 23:36:53,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,<NA>,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2022-01-31 23:44:22,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,<NA>,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2022-01-31 23:39:00,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,<NA>,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2022-01-31 23:36:42,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,<NA>,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN
2022-01-31 23:46:00,2,2022-01-31 23:46:00,2022-02-01 00:13:00,NaN,8.94,NaN,<NA>,186,181,0,25.48,0.0,0.5,6.28,0.0,0.3,35.06,NaN,NaN


In [7]:
ddf.isnull().sum().compute()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

In [9]:
ddf.ffill().compute()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,
2022-01-01 00:35:40,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
2022-01-01 00:33:43,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2022-01-01 00:53:21,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2022-01-01 00:25:21,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
2022-01-01 00:36:48,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 23:36:53,2,2022-01-31 23:36:53,2022-01-31 23:42:51,1.0,1.32,1.0,N,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,2.5,0.0
2022-01-31 23:44:22,2,2022-01-31 23:44:22,2022-01-31 23:55:01,1.0,4.19,1.0,N,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,2.5,0.0
2022-01-31 23:39:00,2,2022-01-31 23:39:00,2022-01-31 23:50:00,1.0,2.10,1.0,N,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,2.5,0.0


In [11]:

ddf.resample('H').agg({'passenger_count': 'sum','fare_amount':'median', 'VendorID': 'count'}).compute()

,passenger_count,fare_amount,VendorID
tpep_pickup_datetime,,,
2008-12-31 22:00:00,1.0,12.0,1
2008-12-31 23:00:00,9.0,7.5,5
2009-01-01 00:00:00,7.0,7.0,5
2009-01-01 01:00:00,0.0,NaN,0
2009-01-01 02:00:00,0.0,NaN,0
...,...,...,...
2022-05-18 16:00:00,0.0,NaN,0
2022-05-18 17:00:00,0.0,NaN,0
2022-05-18 18:00:00,0.0,NaN,0


### optimize for memory

In [12]:
###### Steps ######

In [13]:
ddf = dd.read_parquet("D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-01.parquet", npartitions=1)
s1 = ddf.loc[ddf.tpep_pickup_datetime>'2021-12-31 23:00:00', ['tpep_pickup_datetime','passenger_count','fare_amount', 'VendorID']]
s2 = s1.set_index('tpep_pickup_datetime', sorted=True)
s3 = s2.ffill()
s4 = s3.resample('H').agg({'passenger_count': 'sum', 'VendorID': 'count'})
s4.dask

HighLevelGraph with 10 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x24d9db75840>
 0. read-parquet-91359cce2f3e7050683fa8f19aa28c2d
 1. getitem-9992a3097a956bcaeb5abd688133196a
 2. gt-8d19492220b61c1d58ba9b93cd479533
 3. loc-series-df76065272a8ec02294018bbde8c864d
 4. set_index-1b8054dce8323eb328e06427674de12b
 5. fillna-chunk-4b580a6a90aa6ea498bd83f06c47afab
 6. overlap-concat-eb07456dbdbbaaf73550c41bb6a7288d
 7. overlap-ffill-ea48a997d0a830995b0d5caedb47ab44
 8. repartition-merge-74b350fd1b88944169fe409c5fb5e4db
 9. resample-27b8a0454e7af1dd2e3696a8591a0609

In [14]:
s4.compute().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3310 entries, 2021-12-31 23:00:00 to 2022-05-18 20:00:00
Freq: H
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   passenger_count  3310 non-null   float64
 1   VendorID         3310 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 77.6 KB


In [16]:
def optimeToFitMemory(ddf):
    new_types = dict(
        float32 = ['passenger_count',],
        int16 = ['VendorID',]
    )
    for key in new_types:
        for col in new_types[key]:
            ddf[col] = ddf[col].astype(key)
    return ddf

In [17]:
s5 = s4.map_partitions(optimeToFitMemory)
s5.dask

HighLevelGraph with 11 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x24d9da4ce80>
 0. read-parquet-91359cce2f3e7050683fa8f19aa28c2d
 1. getitem-9992a3097a956bcaeb5abd688133196a
 2. gt-8d19492220b61c1d58ba9b93cd479533
 3. loc-series-df76065272a8ec02294018bbde8c864d
 4. set_index-1b8054dce8323eb328e06427674de12b
 5. fillna-chunk-4b580a6a90aa6ea498bd83f06c47afab
 6. overlap-concat-eb07456dbdbbaaf73550c41bb6a7288d
 7. overlap-ffill-ea48a997d0a830995b0d5caedb47ab44
 8. repartition-merge-74b350fd1b88944169fe409c5fb5e4db
 9. resample-27b8a0454e7af1dd2e3696a8591a0609
 10. optimeToFitMemory-d6d3b2882d64ff924f07e16b2a32e475

In [18]:
s5.compute().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3310 entries, 2021-12-31 23:00:00 to 2022-05-18 20:00:00
Freq: H
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   passenger_count  3310 non-null   float32
 1   VendorID         3310 non-null   int16  
dtypes: float32(1), int16(1)
memory usage: 45.3 KB


In [19]:
df = s5.compute().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3310 entries, 2021-12-31 23:00:00 to 2022-05-18 20:00:00
Freq: H
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   passenger_count  3310 non-null   float32
 1   VendorID         3310 non-null   int16  
dtypes: float32(1), int16(1)
memory usage: 45.3 KB
